In [1]:
import pandas as pd
import numpy as np
import pickle

from scipy.sparse import load_npz, csr_matrix
from sklearn.neighbors import NearestNeighbors
from scripts.caracteristica import caracteristica as car

from datasketch import MinHash, MinHashLSH

In [2]:
def jaccard_distance_sparse(u, v):
    intersection = u.multiply(v).sum()
    union = u.maximum(v).sum()
    return 1.0 - intersection / union

In [3]:
n_perm = 128

def minhash_vector(sparse_vector:csr_matrix, num_perm:int=n_perm):
    minhash = MinHash(num_perm=num_perm)
    for index in sparse_vector.indices:
        minhash.update(str(index).encode('utf8'))
    return minhash

In [4]:
k = 2
th = 0.2
lsh = pickle.load(open(f'../Data/lsh/k{k}_th{th}.pkl', 'rb'))

In [112]:
caracteristica, tweets = car(k=k)

In [113]:
indice = 1007

m = minhash_vector(caracteristica[indice])
aprox = lsh.query(m)

original = tweets.iloc[indice]
print(f'{original["screen_name"] + ";".ljust(10)} {original["text"]}\n')

repeticiones = {}
for i in aprox:
    twt = tweets.iloc[i]
    print( f'{(str(i) + ",").ljust(8)} {twt["screen_name"].strip().ljust(20)}:'.ljust(30), twt['text'] )
    if twt['screen_name'] not in repeticiones:
        repeticiones[twt['screen_name']] = 1
    else:
        repeticiones[twt['screen_name']] += 1

IanDiazc;          la derecha quiere permanecer en la casa del golpista y el cuatro ojos

1349632, patipelada1967      : @baradit @espinozacuellar he visto cosas peores pero en la derecha apruebo sin miramientos este 4 de septiembre
1247234, Patrici77191425     : @renatogaring y tu buscando pega en la derecha con el dedito parado y la lengua en los zapatos que me importa
473094,  micajf2011          : @patriciapolitz y el examen de grado cuando y la recitada en la suprema cuando
532496,  cafermq             : la derecha en la lona
473107,  Ljumberg            : @marcosbarrazag claro la derecha verdad que la derecha es mayoria en la cc tan hipocritas y mentirosos
131093,  Agurt0              : @mvidal_taz @franco_oyarzunv @mdaza_abogado @convencioncl era una propuesta de la derecha que se rechazo en la comision
135198,  Escipio64137610     : @mcubillossigall y la derecha bien gracias salvo honrosas excepciones todas mujeres en la cc y republicanos en el congreso
253995,  ingidsch       

783572,  trueEdgar111        : @berfontaine a falta de argumentos las huevadas sobran en la derecha le dire a medel que te explique
302296,  milenaeldan         : la derecha en la convencion acaba de abstenerse en el articulo que prohibe la desaparicion forzosa de personas el nivel
199909,  FERNAND16166205     : @larusiafacha que mal todo y siento una falta de liderazgo en la derecha todo mal 
615653,  patangachef         : tal cual la derecha solo esta en la convencion para boicotearno quieren aportar
351463,  mariajo09552180     : @berfontaine lo leera el@señor de la carrera y el señor ramirez les metieron el dedo en la boca
963817,  MigUEmm1972         : @patriciapolitz es relevante la votacion de la derecha en la convencion
1133803, CondellJuan         : @berfontaine pero en la derecha no hay expertos ademas la derecha elige puros ignorantes vos mismo
259308,  FabinPonceNez1      : contra la infamia histerica de la derecha aca estan las verdades del pueblo en la 
214256,  lovesport

Seleccionamos tweets de original y el usuario mas repetido.

In [114]:
mas_repetido = max(repeticiones)
tweets = pd.read_parquet('../Data/tweets.parquet')
par_con_nombres = []
par_sin_nombres = []
for c, name in enumerate(tweets['screen_name']):
    if name == original['screen_name']:
        tweet = tweets['text'].iloc[c]
        par_con_nombres.append((name, tweet))
        par_sin_nombres.append(tweet)
    elif name == mas_repetido:
        tweet = tweets['text'].iloc[c]
        par_con_nombres.append((name, tweet))
        par_sin_nombres.append(tweet)

Calculamos shingles

In [115]:
def shingles(k, words: np.ndarray):
    shingles = []
    for i in range(0, len(words)):
        shing = ' '.join(words[i:i+k])
        if len(shing.split()) == k:
            shingles.append(shing)
    return list(set(shingles))


shings_list = []
for twt in par_sin_nombres:
    twt_shings = shingles(2, np.array(twt.split()))
    if twt_shings != []:
        for shing in twt_shings:
            shings_list.append(shing)

Creamos matriz caracteristica entre shingles y tweets de ambas personas

In [116]:
def create_characteristic_matrix(list1, list2):
    matrix = []
    for string1 in list1:
        row = []
        for string2 in list2:
            if string1 in string2:
                row.append(1)
            else:
                row.append(0)
        matrix.append(row)
    return matrix

caracteristica = create_characteristic_matrix(shings_list, par_sin_nombres)

Calculamos diferencia jaccard entre tweets, devuelve posiciones de tweets similares si diferencia jaccard es mayor o igual que s.

In [117]:
def calculate_jaccard_difference(matrix, s):
    num_cols = len(matrix[0])
    jaccard_matrix = []

    for i in range(num_cols):
        for j in range(i + 1, num_cols):
            intersection = np.sum(np.logical_and(matrix[:, i], matrix[:, j]))
            union = np.sum(np.logical_or(matrix[:, i], matrix[:, j]))
            
            if union != 0:
                jaccard = intersection / union
                if jaccard >= s:
                    jaccard_matrix.append([i, j, jaccard])

    return jaccard_matrix

posiciones = calculate_jaccard_difference(np.array(v), 0.3)

In [118]:
for x in posiciones:
    if par_con_nombres[x[0]][0] != par_con_nombres[x[1]][0]:
        print(par_con_nombres[x[0]], par_con_nombres[x[1]])

('IanDiazc', 'la derecha quiere permanecer en la casa del golpista y el cuatro ojos') ('z4ncudoelectric', 'cero esperanza en la derecha tal cual')
('z4ncudoelectric', 'con esto queda toda la derecha inhabilitada ') ('IanDiazc', 'con eso quedan fuera la mitad de la derecha manilarga y raspador a de ollas bien')
('z4ncudoelectric', 'con esto queda toda la derecha inhabilitada ') ('IanDiazc', 'llamen al sr lapiz para que le explique a la derecha del retrazo')
